# RAG graph

In [2]:
%pip install transformers faiss-cpu networkx

Note: you may need to restart the kernel to use updated packages.


In [3]:
import networkx as nx
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

c:\Users\gioel\Desktop\RAG\.venv\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


## Create Graph KB

In [4]:
# Initialize a directed graph
G = nx.DiGraph()

# Add Machine Nodes with Attributes
G.add_node("LaserCutter_1", node_type="Machine", model="LC-200", manufacturer="Brand A")
G.add_node("LaserCutter_2", node_type="Machine", model="LC-300", manufacturer="Brand B")

# Add KPI Nodes with Descriptions and Normal Ranges
G.add_node("WorkingTime", node_type="KPI", description="Time actively working", unit="seconds", normal_min=6, normal_max=10)
G.add_node("IdleTime", node_type="KPI", description="Time idle but available", unit="seconds", normal_min=1, normal_max=4)
G.add_node("OfflineTime", node_type="KPI", description="Time offline and not available", unit="seconds", normal_min=0, normal_max=2)

# Add Directed Relationships (Edges) Between Machines and KPIs
G.add_edge("LaserCutter_1", "WorkingTime", relationship="measures")
G.add_edge("LaserCutter_1", "IdleTime", relationship="measures")
G.add_edge("LaserCutter_1", "OfflineTime", relationship="measures")

G.add_edge("LaserCutter_2", "WorkingTime", relationship="measures")
G.add_edge("LaserCutter_2", "IdleTime", relationship="measures")
G.add_edge("LaserCutter_2", "OfflineTime", relationship="measures")


## Create embeddings for each node

In [5]:
# Function to generate descriptions automatically
def generate_descriptions(graph):
    descriptions = {}

    for node in graph.nodes(data=True):
        node_name, attributes = node
        node_type = attributes.get("node_type")

        if node_type == "Machine":
            # Find KPIs measured by this machine
            kpis = [target for source, target, data in graph.edges(data=True) if source == node_name and data["relationship"] == "measures"]
            kpi_list = ", ".join(kpis)

            # Generate description using template
            descriptions[node_name] = (
                f"Machine type {attributes.get('node_type')}, model {attributes.get('model')}, "
                f"manufactured by {attributes.get('manufacturer')}. Measures KPIs: {kpi_list}."
            )

        elif node_type == "KPI":
            # Find machines that measure this KPI
            machines = [source for source, target, data in graph.edges(data=True) if target == node_name and data["relationship"] == "measures"]
            machine_list = ", ".join(machines)

            # Generate description using template
            descriptions[node_name] = (
                f"KPI {node_name}, measures {attributes.get('description')} in {attributes.get('unit')}. "
                f"Normal range is {attributes.get('normal_min')} to {attributes.get('normal_max')}. "
                f"Measured by machines: {machine_list}."
            )

    return descriptions

# Generate and print descriptions for all nodes
generated_descriptions = generate_descriptions(G)
for node, desc in generated_descriptions.items():
    print(f"{node}: {desc}")

LaserCutter_1: Machine type Machine, model LC-200, manufactured by Brand A. Measures KPIs: WorkingTime, IdleTime, OfflineTime.
LaserCutter_2: Machine type Machine, model LC-300, manufactured by Brand B. Measures KPIs: WorkingTime, IdleTime, OfflineTime.
WorkingTime: KPI WorkingTime, measures Time actively working in seconds. Normal range is 6 to 10. Measured by machines: LaserCutter_1, LaserCutter_2.
IdleTime: KPI IdleTime, measures Time idle but available in seconds. Normal range is 1 to 4. Measured by machines: LaserCutter_1, LaserCutter_2.
OfflineTime: KPI OfflineTime, measures Time offline and not available in seconds. Normal range is 0 to 2. Measured by machines: LaserCutter_1, LaserCutter_2.


In [6]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import faiss

# Load the Sentence Transformer model for embedding-based retrieval
embedder = SentenceTransformer("all-MiniLM-L6-v2")

# Assuming `generated_descriptions` contains descriptions for each node
node_embeddings = {node: embedder.encode(description) for node, description in generated_descriptions.items()}

# Convert node embeddings to a matrix for FAISS
embeddings_matrix = np.array(list(node_embeddings.values())).astype("float32")
embedding_dim = embeddings_matrix.shape[1]

# Initialize FAISS index with the correct dimension
index = faiss.IndexFlatL2(embedding_dim)
index.add(embeddings_matrix)

# Map node names to their index positions in FAISS
node_to_id = {node: idx for idx, node in enumerate(node_embeddings.keys())}
id_to_node = {idx: node for node, idx in node_to_id.items()}

# Function to retrieve top-k nodes using SentenceTransformer embeddings and FAISS
def retrieve_top_k_nodes(query, top_k=3):
    query_embedding = embedder.encode(query).reshape(1, -1).astype("float32")
    distances, indices = index.search(query_embedding, top_k)
    retrieved_nodes = [
        {
            "node": id_to_node[idx],
            "description": generated_descriptions[id_to_node[idx]],
            "distance": distances[0][i]
        }
        for i, idx in enumerate(indices[0])
    ]
    return retrieved_nodes

# Set up the FLAN-T5 model for text generation
model_name = "google/flan-t5-large"  # or "google/flan-t5-xl"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a text generation pipeline with FLAN-T5
generator_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    device=0 if torch.cuda.is_available() else -1,
    max_new_tokens=100,
    do_sample=True
)

# Example query for retrieval
query = "How many LaserCutting machines are there?"
retrieved_nodes = retrieve_top_k_nodes(query)

# Combine descriptions into a single context string
context = "\n".join([f"{node['node']}: {node['description']}" for node in retrieved_nodes])

# Format the prompt with the context and query for the FLAN-T5 model
prompt = f"""
Please answer the following question based on the provided information. Offer a clear and thorough explanation that directly answers the question and includes any important details or context for a better understanding.

Question: {query}

Additional information:
{context}

Answer:
"""

# Generate the response
response = generator_pipeline(prompt)
print("Response:", response[0]["generated_text"])

Response: The total number of LaserCutting machines in the country is .


In [7]:
print(context)

WorkingTime: KPI WorkingTime, measures Time actively working in seconds. Normal range is 6 to 10. Measured by machines: LaserCutter_1, LaserCutter_2.
IdleTime: KPI IdleTime, measures Time idle but available in seconds. Normal range is 1 to 4. Measured by machines: LaserCutter_1, LaserCutter_2.
LaserCutter_2: Machine type Machine, model LC-300, manufactured by Brand B. Measures KPIs: WorkingTime, IdleTime, OfflineTime.


In [16]:
# Funzione per calcolare precision@k e recall@k
def evaluate_retrieval(query, relevant_nodes, top_k=3):
    # Recupera i top-k nodi per la query
    retrieved_nodes = retrieve_top_k_nodes(query, top_k=top_k)
    retrieved_node_names = [node["node"] for node in retrieved_nodes]

    # Genera un vettore binario per nodi rilevanti e recuperati
    relevant_set = set(relevant_nodes)
    retrieved_set = set(retrieved_node_names)
    
    print("relevant set: "+relevant_set.__str__())
    print("retrieved set: "+retrieved_set.__str__())

    # Precision@k: quanti dei recuperati sono rilevanti
    precision_at_k = len(relevant_set & retrieved_set) / top_k

    # Recall@k: quanti dei rilevanti totali sono stati recuperati
    recall_at_k = len(relevant_set & retrieved_set) / len(relevant_set)

    return precision_at_k, recall_at_k

# Esempio di utilizzo
query = "How many LaserCutting machines are there?"
relevant_nodes = ["LaserCutter_1", "LaserCutter_2"]
precision, recall = evaluate_retrieval(query, relevant_nodes)
print(f"Precision@k: {precision:.2f}, Recall@k: {recall:.2f}")


relevant set: {'LaserCutter_2', 'LaserCutter_1'}
retrieved set: {'WorkingTime', 'IdleTime', 'LaserCutter_2'}
Precision@k: 0.33, Recall@k: 0.50


In [12]:
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from nltk.translate.meteor_score import meteor_score

# Funzione per valutare la qualità della generazione
def evaluate_generation(generated_response, reference_response):
    # Tokenizziamo il testo per meteor_score
    reference_tokenized = reference_response.split()
    generated_tokenized = generated_response.split()

    # BLEU
    bleu_score = sentence_bleu([reference_tokenized], generated_tokenized)
    
    # ROUGE
    rouge = Rouge()
    rouge_scores = rouge.get_scores(generated_response, reference_response, avg=True)
    
    # METEOR
    meteor = meteor_score([reference_tokenized], generated_tokenized)
    
    return {
        "BLEU": bleu_score,
        "ROUGE": rouge_scores,
        "METEOR": meteor
    }

# Esempio di utilizzo
reference_response = "There are 2 laser cutting machines."
generated_response = response[0]["generated_text"]

scores = evaluate_generation(generated_response, reference_response)
print("Evaluation Scores:", scores)

Evaluation Scores: {'BLEU': 0, 'ROUGE': {'rouge-1': {'r': 0.16666666666666666, 'p': 0.1, 'f': 0.12499999531250018}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.16666666666666666, 'p': 0.1, 'f': 0.12499999531250018}}, 'METEOR': 0.0}
